In [1]:
!pip install tensorflow
#check 123

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import os
import random
import numpy as np
import shutil
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from sklearn.model_selection import train_test_split
import requests
import tarfile
import os

In [10]:
def download_file(url, filename):
    response = requests.get(url, stream=True)
    with open(filename, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)

def extract_tarfile(filename, path):
    with tarfile.open(filename, "r:gz") as tar_ref:
        tar_ref.extractall(path)

In [11]:
# Add the download and extraction code from the previous answer

data_url = "http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz"
filename = "food-101.tar.gz"

download_file(data_url, filename)
extract_tarfile(filename, ".")

# Prepare the dataset with only 3 classes and 500 images
source_dir = "food-101/images/"
dest_dir = "reduced_food-101/"

os.makedirs(dest_dir, exist_ok=True)

classes = os.listdir(source_dir)
selected_classes = random.sample(classes, 3)

for c in selected_classes:
    os.makedirs(os.path.join(dest_dir, c), exist_ok=True)
    class_images = os.listdir(os.path.join(source_dir, c))
    selected_images = random.sample(class_images, min(500, len(class_images)))
    for img in selected_images:
        shutil.copy(os.path.join(source_dir, c, img), os.path.join(dest_dir, c, img))

In [12]:
train_dir = "train/"
val_dir = "val/"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

for c in selected_classes:
    os.makedirs(os.path.join(train_dir, c), exist_ok=True)
    os.makedirs(os.path.join(val_dir, c), exist_ok=True)
    images = os.listdir(os.path.join(dest_dir, c))
    train_images, val_images = train_test_split(images, test_size=0.2)

    for img in train_images:
        shutil.move(os.path.join(dest_dir, c, img), os.path.join(train_dir, c, img))
    for img in val_images:
        shutil.move(os.path.join(dest_dir, c, img), os.path.join(val_dir, c, img))

In [13]:
BATCH_SIZE = 32
IMG_SIZE = (299, 299)

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode="nearest",
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
)

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [14]:
# Create the base model
base_model = Xception(
    weights="imagenet", include_top=False, input_shape=IMG_SIZE + (3,)
)

# Freeze the base model
base_model.trainable = False

# Add a custom classification head
inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(selected_classes), activation="softmax")(x)

model = models.Model(inputs, outputs)

# Compile the model
model.compile(optimizer=optimizers.Adam(lr=0.001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

83683744/83683744 [==============================] - 0s 0us/step


In [15]:
EPOCHS = 10

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
)

Epoch 1/10
37/37 [==============================] - 463s 12s/step - loss: 0.2832 - accuracy: 0.8853 - val_loss: 0.1557 - val_accuracy: 0.9444
Epoch 2/10
37/37 [==============================] - 452s 12s/step - loss: 0.1358 - accuracy: 0.9469 - val_loss: 0.1224 - val_accuracy: 0.9444
Epoch 3/10
37/37 [==============================] - 455s 12s/step - loss: 0.1290 - accuracy: 0.9452 - val_loss: 0.0825 - val_accuracy: 0.9653
Epoch 4/10
37/37 [==============================] - 509s 14s/step - loss: 0.1002 - accuracy: 0.9649 - val_loss: 0.0884 - val_accuracy: 0.9583
Epoch 5/10
37/37 [==============================] - 463s 12s/step - loss: 0.0955 - accuracy: 0.9598 - val_loss: 0.1008 - val_accuracy: 0.9618
Epoch 6/10
37/37 [==============================] - 468s 13s/step - loss: 0.0752 - accuracy: 0.9735 - val_loss: 0.0827 - val_accuracy: 0.9688
Epoch 7/10
37/37 [==============================] - 471s 13s/step - loss: 0.0648 - accuracy: 0.9743 - val_loss: 0.0868 - val_accuracy: 0.9688
Epoch 

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a lower learning rate
model.compile(optimizer=optimizers.Adam(lr=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Fine-tune the model
fine_tune_epochs = 10
total_epochs = EPOCHS + fine_tune_epochs

history_fine_tune = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
)

Epoch 10/20
37/37 [==============================] - 1390s 37s/step - loss: 1.4679 - accuracy: 0.3433 - val_loss: 1.0986 - val_accuracy: 0.3368
Epoch 11/20
32/37 [========================>.....] - ETA: 2:50 - loss: 1.0990 - accuracy: 0.3165